<a href="https://colab.research.google.com/github/yezzzzin/object-detectinon/blob/main/data_preprocess.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
from scipy.fftpack import fft
from scipy.signal import get_window
from scipy.fftpack import dct

In [ ]:
def compute_mfcc(audio_signal, sample_rate=44100, frame_size=0.025, frame_stride=0.01, num_mfcc=13, nfilt=26, nfft=512):
    # Step 1: Frame Segmentation
    frame_length, frame_step = frame_size * sample_rate, frame_stride * sample_rate
    signal_length = len(audio_signal)
    frame_length = int(round(frame_length))
    frame_step = int(round(frame_step))
    num_frames = int(np.ceil(float(np.abs(signal_length - frame_length)) / frame_step))

    # zero padding
    pad_signal_length = num_frames * frame_step + frame_length
    z = np.zeros((pad_signal_length - signal_length))
    pad_signal = np.append(audio_signal, z)

    indices = np.tile(np.arange(0, frame_length), (num_frames, 1)) + np.tile(np.arange(0, num_frames * frame_step, frame_step), (frame_length, 1)).T
    frames = pad_signal[indices.astype(np.int32, copy=False)]

    # Step 2: FFT
    mag_frames = np.absolute(fft(frames, nfft))[:nfft // 2]

    # Step 3: Mel filterbank
    mel_filterbank = mel_filterbank_coefficients(sample_rate, nfilt, nfft)
    mel_energy = np.dot(mag_frames, mel_filterbank.T)
    mel_energy = np.where(mel_energy == 0, np.finfo(float).eps, mel_energy)  # Avoiding zero

    # Step 4: Log transform
    mel_energy = np.log(mel_energy)

    # Step 5: DCT
    mfcc = dct(mel_energy, type=2, axis=1, norm='ortho')[:, 1 : (num_mfcc + 1)]

    return mfcc

In [ ]:
def mel_filterbank_coefficients(sample_rate, nfilt, nfft):
    low_freq_mel = 0
    high_freq_mel = (2595 * np.log10(1 + (sample_rate / 2) / 700))
    mel_points = np.linspace(low_freq_mel, high_freq_mel, nfilt + 2)
    hz_points = (700 * (10**(mel_points / 2595) - 1))
    bin_points = np.floor((nfft + 1) * hz_points / sample_rate)

    fbank = np.zeros((nfilt, nfft // 2 + 1))
    for m in range(1, nfilt + 1):
        f_m_minus = int(bin_points[m - 1])
        f_m = int(bin_points[m])
        f_m_plus = int(bin_points[m + 1])
        for k in range(f_m_minus, f_m):
            fbank[m - 1, k] = (k - bin_points[m - 1]) / (bin_points[m] - bin_points[m - 1])
        for k in range(f_m, f_m_plus):
            fbank[m - 1, k] = (bin_points[m + 1] - k) / (bin_points[m + 1] - bin_points[m])
    return fbank